# Anchor explanations for income prediction

In this example, we will explain predictions of a Random Forest classifier whether a person will make more or less than $50k based on characteristics like age, marital status, gender or occupation. The features are a mixture of ordinal and categorical data and will be pre-processed accordingly.

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult

%load_ext autoreload
%autoreload 2


/home/robert/anaconda3/envs/alibi/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load adult dataset

The `fetch_adult` function returns a `Bunch` object containing the features, the targets, the feature names and a mapping of categorical variables to numbers which are required for formatting the output of the Anchor explainer.

In [2]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [3]:
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

Note that for your own datasets you can use our utility function [gen_category_map](../api/alibi.utils.data.rst) to create the category map:

In [4]:
from alibi.utils.data import gen_category_map

Define shuffled training and test set

In [5]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:,:-1]
target = data_perm[:,-1]

In [6]:
idx = 30000
X_train,Y_train = data[:idx,:], target[:idx]
X_test, Y_test = data[idx+1:,:], target[idx+1:]

### Create feature transformation pipeline
Create feature pre-processor. Needs to have 'fit' and 'transform' methods. Different types of pre-processing can be applied to all or part of the features. In the example below we will standardize ordinal features and apply one-hot-encoding to categorical features.

Ordinal features:

In [7]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())])

Categorical features:

In [8]:
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Combine and fit:

In [9]:
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features),
                                               ('cat', categorical_transformer, categorical_features)])
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 [0, 8, 9, 10]),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [1, 2, 3, 4, 5, 6, 7, 11])])

### Train Random Forest model

Fit on pre-processed (imputing, OHE, standardizing) data.

In [10]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(preprocessor.transform(X_train), Y_train)

RandomForestClassifier(n_estimators=50)

Define predict function

In [11]:
predict_fn = lambda x: clf.predict(preprocessor.transform(x))
print('Train accuracy: ', accuracy_score(Y_train, predict_fn(X_train)))
print('Test accuracy: ', accuracy_score(Y_test, predict_fn(X_test)))

Train accuracy:  0.9655333333333334
Test accuracy:  0.855859375


### Initialize and fit anchor explainer for tabular data

In [74]:
explainer = AnchorTabular(predict_fn, feature_names, categorical_names=category_map, seed=1)

Discretize the ordinal features into quartiles

### Getting an anchor

Below, we get an anchor for the prediction of the first observation in the test set. An anchor is a sufficient condition - that is, when the anchor holds, the prediction should be the same as the prediction for this instance.

In [86]:
X_test[6], adult.feature_names

(array([  48,    6,    4,    0,    2,    0,    4,    1,    0, 1902,   40,
           9]),
 ['Age',
  'Workclass',
  'Education',
  'Marital Status',
  'Occupation',
  'Relationship',
  'Race',
  'Sex',
  'Capital Gain',
  'Capital Loss',
  'Hours per week',
  'Country'])

In [133]:
idx = 6
class_names = adult.target_names
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

disc = np.linspace(0, 100, 5).astype(np.int32)
explainer.fit(X_train, disc_perc=11)
explanation = explainer.explain(X_test[idx], threshold=0.95, verbose=False)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Prediction:  >50K
Anchor: 1742.40 < Capital Loss <= 2178.00 AND Relationship = Husband AND 38.90 < Age <= 53.50 AND Race = White AND Country = United-States AND 30.40 < Hours per week <= 69.60 AND Capital Gain <= 99999.00 AND Sex = Male AND Marital Status = Married AND Workclass = Self-emp-not-inc
Precision: 0.95
Coverage: 0.00


In [131]:
ft_idx = adult.feature_names.index('Capital Gain')
qts = np.array(np.percentile(X_train[:, ft_idx], [25, 50, 75]))
print(qts)

[0. 0. 0.]


In [130]:
ft_idx = adult.feature_names.index('Capital Gain')
qts = np.array(np.percentile(X_train[:, ft_idx], [10, 20, 30, 40, 50, 60, 70, 80, 90]))
print(qts)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [132]:
np.min(X_train[:, ft_idx]), np.max(X_train[:, ft_idx])

(0, 99999)